# Semi-supervised Labeling

<img src="assets/SampledDistribution.jpg" width="500"/>

Kernel density estimate are used to systematically identify bone possible regions - called class 1's

**class 0** 
- Non-Bone samples are relatively easy to identify and are common throughout the globe and common in regions which are within a few miles of bone sites

**class 2** 
- Bone locations are identified via GPS locations and mapped to image coordinates

**class 1**
- any region with similar depositional environments and for which bones have not been positively identified. These are different in look and feel, visual texture and color, from Non-Bone locations


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Load Kernel Density Map

Load the density map from the trainin location in central New Mexico

In [ ]:
z = np.load('data/z.npy')
meshParams = np.load('data/meshParams.npy')
xx = np.load('data/xx.npy')
yy = np.load('data/yy.npy')
xmin, xmax, ymin, ymax, inc = meshParams

# Plot histogram of  bone likelihoods

The data used to generate this KDE consisted of over 5000 bone sample locations from central New Mexico.

These histograms show MANY more locations that this! Can you explain how we get such large multiplicative factors of label data through this sampling technique?

How can there be so many counts in each bin?  We don’t have anywhere near that many fossil locations.

The reason is that the meshgrid we created has hundreds of thousands of locations. Sampling from this gives us many possible samples


In [ ]:
print('number of grid locations: ', z.shape)
counts, binloc = np.histogram(z, bins = 4)
labs = ['{:3.1f}'.format(n) for n in np.round_(binloc[:-1], 1) ]


fig, ax = plt.subplots()
ax.bar(labs, counts)
plt.title('Histogram of bone likelyhood')
#ax.set_yscale('log')
ax.set_xlabel('bins')
ax.set_ylabel('counts')
plt.grid()

plt.show()
print("bin range is:\t\t", labs)
print("Counts in each bin:\t", counts)

# Create mapping from image coordinates to meshgrid index

We use an increment = inc for the meshgrid which offers a coarse granularity of cell choices to speed up and shrink memory usage for the meshgrid. Setting inc to 1 gives finest granularity.

the Cell Index function is the means to linearly map from Image coordinates to messgrid cell coordinates

Image coordinates:
- xx ranges from 0 to 4160 pixels
- yy ranges from 9 to 5257 pixels

meshgrid cells are indexed differently
- CellX ranges from 0 to 260
- CellY ranges from 0 to 328

Create simple linear regression prediction function to map the two

We define **CellIndex( Image_coordinates_x, Image_coordinates_y)**:
- return messgrid_xx, meshgrid_yy

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

CellX = LinearRegression().fit(xx[:,0].reshape(-1, 1), [i for i in range(xx.shape[0])])
CellY = LinearRegression().fit(yy[0,:].reshape(-1, 1), [ymax//inc - i for i in range(1 + ymax//inc)] )

def CellIndex(x, y):
    X = np.array([x]).reshape(-1,1)
    Y = np.array([y]).reshape(-1,1)
    cellx = int(CellX.predict(X))
    celly = yy.shape[1] - int(CellY.predict(Y)[0] +.5) 
    return cellx, celly

idx = CellIndex(2000, 2000)
print(idx)
z[idx]

## Print extremes of Cell Index

It seems upside down because images coordinates in the Y axis are always 0 at the TOP of the image

In [ ]:
print(xmax, ymax)
a,b = xmin,ymin; print("CellIndex({}, {}) -> {}".format(a, b, CellIndex(a,b)))
a,b = xmin,ymax; print("CellIndex({}, {}) -> {}".format(a, b, CellIndex(a,b)))
a,b = xmax,ymin; print("CellIndex({}, {}) -> {}".format(a, b, CellIndex(a,b)))
a,b = xmax,ymax; print("CellIndex({}, {}) -> {}".format(a, b, CellIndex(a,b)))

# Plot the kernel density map

This is correlated to the dimensions of the image

Plot a green dot at various locates to validate the density plot is accurate

This is how we will label data!

Pick meshgrid (to get a 224x224 sized image, compute its CellIndex

- X,Y = (2800,2800)
- z[CellIndex(X,Y)]

yields 0.36899990820120526



In [ ]:
X_LL, Y_LL = (2000,2000)
print('(2000,2000) -> ',  z[CellIndex(X_LL, Y_LL)])

X_UR, Y_UR = (2224,2224)
print('(2224,2224) -> ',  z[CellIndex(X_UR, Y_UR )])

X_mean, Y_mean = (X_LL + X_UR)//2, (Y_LL + Y_UR)//2
print('X_mean, Y_mean: ', (X_mean, Y_mean))

z[CellIndex(X_mean, Y_mean)]

# So a cropped image with coordinates X_LL, Y_UR, will be scored using z
# and the value for z[(2112, 2112)] is 0.15808224823133873
# so that cropped image will be named something like:
# 'DinoX2000Y2000' so we know where to glue it back into a reconstructed map
# it gets scored as .15808 which we round to .1 and the image is moved to the ".1" folder
# We label the image by putting it nto a folder with names:
# 0, .1, .2, ...., .9

# Plot KDE with test location

Test location in lime color - specify coordinate, plot the location and sample from the KDE at that location.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.gca()
# xmax, ymax = 10100, 10256
# xmin, ymin = 0,0
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymax, ymin )
cfset = ax.contourf(xx, yy, z, cmap='coolwarm', levels=[i/20 for i in range (20)], alpha =.2)
#cfset = ax.contourf(xx, yy, z, cmap='coolwarm', levels= 3, alpha =.2)
ax.imshow(np.rot90(z), cmap='coolwarm', extent=[xmin, xmax, ymax, ymin  ])

#cset = ax.contour(xx, yy, z, colors='k')
cset = ax.contour(xx, yy, z, levels = 5, colors='k')
ax.clabel(cset, inline=1, fontsize=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.title('2D Gaussian Kernel density estimation')

# plot a sample piont in lime and read it density
X,Y = (2112, 2112)
X,Y = (2250, 2900)
plt.scatter(X, Y,c='lime')
plt.grid()
idx = CellIndex(X, Y)
print("Image coordinate: ", (X,Y))
print("Meshgrid Index coordinate: ", idx)
print('Likelyhood: ', z[idx])

# Plot Sampling Label points

Separate all the sample points so that z values fall into 4 bins

Sample each of the 4 bins with varying probabilities to keep the sampling densities roughly the same. This is to keep labels roughly balanced.

Of the 4 bins: 

- 0 is the lowest probability of bone 
- 1 is the next higher bone probability 
- 2 higher 
- 3 highest

Empirically, I made the observation that simply using these density plots yields very poor model.


### class labels:
- **class 0** are places on the planet where you can confidently assert that no dinosaur bones are present
- **class 1** are places where bones are likely to be found - the depositional environment is favorable, but to date not bones have been observed there. Class 1 is the one estimated from the Kernel density plot.
- **class 2** are places where there are definitive waypoints with known bone sites


Each label probability of choice is individually controllable via the variable **choice**


In [ ]:
# find the indices for all z values less than .1 we will consider these class 0

choice = [2000, 1500, 450, 150]  # How many from each class to sample

rng = np.random.default_rng(2021)

dataL = []
bins = 4
for labelDec in range(bins):
    label = labelDec/bins  # proportional multiple of 1/bins [0, 1/4, 2/4, 3/4]
    idx_0, idy_0 = np.where((z >= label)  &  (z < label + 1/bins))
    # identify all the places where z >= label
    # store the places as idx_0, idy_0
    # choice[0] = 2000, choice[1] = 1500, choice[2] = 450, etc...
    # rng.choice selectsthe idx_0 indices according to specified sampling choice
    id = rng.choice( [i for i in range(idx_0.shape[0])], choice[labelDec]) 
    for i in range(choice[labelDec]):
        X = idx_0[id[i]] # randomly sampled X values indexed by bin (specific value of z)
        Y = idy_0[id[i]] # randomly sampled Y values indexed by bin
        dataL.append([xx[X,Y], yy[X,Y], label*bins])
        data = np.array(dataL).astype(int)
print("Done")

## Plot a sample image slice of size 224, 224

This is one known bone location from Central New Mexico
- left   = 1550
- right  = left + 224
- top    = 3250
- bottom = top + 224

In [ ]:
left   = 1550
right  = left + 224
top    = 3250
bottom = top + 224

from PIL import Image
CO = 'data/SomeWhereColorado.png'
img = Image.open(CO)

im1 = img.crop((left, top, right, bottom))
print(im1.size)
im1.show()

# Sample Thousands of Points from KDE

Color them by bin level (4 bins) associated with contours of z

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12, 12), dpi=80)


ax.scatter(data[:,0], data[:,1].max() - data[:,1], c = data[:,2], s=4, cmap='viridis')
#plt.colorbar()
ax.legend()
ax.grid(True)
plt.show()

# Write Image Patches to Train & Val folders

## # Special 4 Bin manually reassigned to 1

Based on subject matter expertise, the bin 1 appears to be the best choice for calling the label class 1 (bone possible)

Bin 0 cannot reliably predict NON Bone (the probabilities are still too high)

Bin 3 cannot reliably predict bone as the smoothness or rounding of the surface provided by KDE is too broad and encompasses too many bone as well as Non bone regions. This can be controlled by the BW parameter but wins in getting better bone predictions by adjusting BW come at expense of LOSS of predictability of bone possible class 1.

Move train 0 to class train 0
Move y_train > train 0 to class train 1
Copy reduced/train/NoBone... to class train 0
Copy reduced/train/Bone... to class train 2

Move val 0 to class val 0
Move y_val > val 0 to class val 1
Copy reduced/val/NoBone... to class val 0
Copy reduced/val/Bone... to class val 2

### Running Comment out cell below - will overwrite data folder

Do so with specific intention

The cell below has been commented out by selecting all lines and pressing CTRL /

To run the cell: UNCOMMENT the entire cell - but realize this will write new images to the folder which need cleaning prior to training.

This will create class 1 partial images that contain black regions of pixels that will negatively affect training. These images must be removed with a utility function.



In [ ]:
# import time
# tStart = time.time()
# import numpy as np
# from sklearn.model_selection import train_test_split
# from tqdm import tqdm 

# data = np.array(data).astype(int)

# X_train, X_val, y_train, y_val = train_test_split(
#     data[:,:2], data[:,2], test_size=0.2, random_state=42)
# for i in tqdm(range(len(y_train))):

#     fn = "data/ThreeClassBalanced5000/train/{}/NMCentralX{:04d}Y{:04d}.png".format(int(y_train[i]), X_train[i,0], X_train[i][1])
#     left   = data[i][0]
#     right  = left + 224
#     top    = data[i][1]
#     bottom = top + 224
#     im1 = img.crop((left, top, right, bottom))
#     im1.save(fn)
#     im1.close()
#     #print(fn)
# print("Train Folder: Elapsed: ", tEnd - tStart)      
# for i in tqdm(range(len(y_val))):
#     if label != int(y_train[i]):
#         print(int(y_train[i]))
#     label = int(y_train[i])
#     fn = "data/ThreeClassBalanced5000/val/{}/NMCentralX{:04d}Y{:04d}.png".format(y_val[i], X_val[i][0], X_val[i][1])
#     left   = data[i][0]
#     right  = left + 224
#     top    = data[i][1]
#     bottom = top + 224
#     im1 = img.crop((left, top, right, bottom))
#     im1.save(fn)
#     im1.close()
#     #print(fn)

# # remove any .ipynb_checkpoint folders
# ext = '*.ipynb_checkpoint'
# for fn in glob.glob(Folder + ext):
#     os.remove(fn, dir_fd=None)
# F = 'data/reducedPNG/train/BoneAerialImagesReducedGE'
# for fn in glob.glob(Folder + ext):
#     os.remove(fn, dir_fd=None)
# F = 'data/reducedPNG/train/NoBonesAerialImagesReducedGE'
# for fn in glob.glob(Folder + ext):
#     os.remove(fn, dir_fd=None)
# F = 'data/reducedPNG/val/BoneAerialImagesReducedGE'
# for fn in glob.glob(Folder + ext):
#     os.remove(fn, dir_fd=None)
# F = 'data/reducedPNG/val/NoBonesAerialImagesReducedGE'
# for fn in glob.glob(Folder + ext):
#     os.remove(fn, dir_fd=None)
    
# tEnd = time.time()
# print("Train & Val: Elapsed ", tEnd - tStart)  

In [ ]:
# !du --inodes data/ThreeClassBalanced5000/

In [ ]:
# !rm -rf data/ThreeClassBalanced5000/train/0/.ipynb_checkpoints/
# !rm -rf data/ThreeClassBalanced5000/train/1/.ipynb_checkpoints/
# !rm -rf data/ThreeClassBalanced5000/train/2/.ipynb_checkpoints/

# !rm -rf data/ThreeClassBalanced5000/val/0/.ipynb_checkpoints/
# !rm -rf data/ThreeClassBalanced5000/val/1/.ipynb_checkpoints/
# !rm -rf data/ThreeClassBalanced5000/val/2/.ipynb_checkpoints/


If you have any issues or want to contribute, please contact our authors:
Intel oneAPI Solution Architect
- Chesebrough, Bob [bob.chesebrough (at) intel.com]
